In [8]:
%pip install geopandas


     |████████████████████████████████| 323 kB 3.2 MB/s eta 0:00:01
     |████████████████████████████████| 4.9 MB 65.5 MB/s eta 0:00:01
     |████████████████████████████████| 10.8 MB 77.2 MB/s eta 0:00:01
     |████████████████████████████████| 19.5 MB 38.0 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 79.9 MB/s eta 0:00:01
     |████████████████████████████████| 5.3 MB 73.9 MB/s eta 0:00:01
     |████████████████████████████████| 509 kB 95.2 MB/s eta 0:00:01
     |████████████████████████████████| 347 kB 83.7 MB/s eta 0:00:01
     |████████████████████████████████| 159 kB 90.4 MB/s eta 0:00:01
You should consider upgrading via the '/Users/moriseiitsu/Dropbox/IDEAMAPS/ideamaps-models/models/emergency-maternal-care/scripts/.venv_requirements_test/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
# Set paths to access data
# Define directories
data_inputs = './Nairobi/data-inputs/'
data_temp = './Nairobi/data-temp/'
data_outputs = '../nairobi/'

In [9]:
import geopandas as gpd
import os
healthcare_facilities_validated = gpd.read_file(data_inputs + 'helthcare-facilities.geojson')

In [10]:
healthcare_facilities_validated

,fid,field_1_1,Admin1,Facility_n,Facility_t,Ownership,Lat,Long,LL_source,Ward,Sub County,Level,Offer EmOC (Y/N),Type of EmOC (basic/comprehensive),Total In-patient beds,Maternity beds,Maternity theatres,Operating time,Comments,geometry
0,1,Kenya,Nairobi,Dandora II Health Centre,Health Centre,Local authority,-1.257610,36.887750,GPS,Dandora Area III,Embakasi North,Level 3,True,BMOC,15,N/A,N/A,24hrs,None,POINT (36.88775 -1.25761)
1,2,Kenya,Nairobi,Embakasi Health Centre,Health Centre,Local authority,-1.306200,36.914700,GPS,Embakasi,Embakasi East,Level 3,True,CeMOC & BMOC,N/A,N/A,N/A,24hrs,None,POINT (36.9147 -1.3062)
2,3,Kenya,Nairobi,Imara Health Centre,Health Centre,FBO,-1.324300,36.888700,Google Earth,Kware,Embakasi South,Level 3,True,BMOC,6,N/A,N/A,24hrs & weekends,None,POINT (36.8887 -1.3243)
3,4,Kenya,Nairobi,Jonalifa Clinic,Clinic,NGO,-1.285900,36.739880,Google Earth,Kawangware,Dagoretti South,Level 2,True,BMOC,8,N/A,N/A,24hrs & weekends,None,POINT (36.73988 -1.2859)
4,5,Kenya,Nairobi,Kangemi Health Centre,Health Centre,Local authority,-1.264600,36.749300,GPS,Kitisuru,Westlands,Level 3,True,BMOC,N/A,N/A,N/A,24hrs,None,POINT (36.7493 -1.2646)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,150,Kenya,Nairobi,Nairobi Hospital,Comprehensive Teaching & Tertiary Referral Hos...,Private,-1.294743,36.800247,KMHFR,Kilimani,Dagoretti North,Level 5,True,CeMOC&BMOC,350,N/A,N/A,24hrs+weekends,None,POINT (36.80025 -1.29474)
149,151,Kenya,Nairobi,Al-Basrah Hospital Limited,Primary care hospitals,Private,-1.255881,36.876857,KMHFR,Kiamaiko,Mathare,Level 4,True,BMOC,N/A,10,1,24hrs+weekends,None,POINT (36.87686 -1.25588)
150,152,Kenya,Nairobi,St. Oswald Medical Centre,Medical Center,Private,-1.290367,36.729885,KMHFR,Riruta,Dagoretti South,Level 3,True,BMOC,12,2,N/A,24hrs+weekends,None,POINT (36.72989 -1.29037)
151,153,Kenya,Nairobi,Nairobi West Hospital,Primary Care hospitals,Private,-1.306840,36.825760,KMHFR,Nairobi West,Langata,Level 4,True,BMOC,152,N/A,N/A,24hrs+weekends,None,POINT (36.82576 -1.30684)


In [1]:
import numpy as np

In [11]:
conditions = [
    healthcare_facilities_validated['specific_owner'].isin([1, 3, 6]),
    healthcare_facilities_validated['specific_owner'].isin([2, 4, 5])
]
choices = ['Public Comprehensive EmOC', 'Private Comprehensive EmOC']

# Explicitly set the default to a string
healthcare_facilities_validated['Validation'] = np.select(
    conditions, choices, default='Other'
)

KeyError: 'specific_owner'

In [ ]:
study_area = gpd.read_file(data_inputs + '100mGrid.gpkg')
raster_path = data_inputs + 'nga_f_15_49_2015_1km.tif'

In [2]:
pip install rasterio
import rasterio
from rasterio.mask import mask

ModuleNotFoundError: No module named 'rasterio'

In [ ]:
with rasterio.open(raster_path) as dataset:
    geometries = [study_area.geometry.unary_union.__geo_interface__]
    clipped_image, clipped_transform = mask(dataset, geometries, crop=True)
    band1 = clipped_image[0] # Read the first band of the raster

In [ ]:
out_meta = dataset.meta.copy()
out_meta.update({
        "height": clipped_image.shape[1],
        "width": clipped_image.shape[2],
        "transform": clipped_transform
    })

In [ ]:
with rasterio.open(data_inputs + 'nairobi_nga_f_15_49_2015_1km.tif', "w", **out_meta) as dest:
    dest.write(clipped_image)

In [ ]:
rows, cols = np.where(band1 > 0)
grid_cells = [clipped_transform * (col + 0.5, row + 0.5) for row, col in zip(rows, cols)]
population_values = band1[rows, cols]

In [3]:
from shapely.geometry import Point
import pandas as pd

In [ ]:
grid_df = pd.DataFrame(grid_cells, columns=["longitude", "latitude"])
grid_df["population"] = population_values

grid_df["grid_code"] = range(len(grid_df))
population_centroids_gdf = gpd.GeoDataFrame(grid_df, geometry=[Point(xy) for xy in zip(grid_df["longitude"], grid_df["latitude"])])
population_centroids_gdf.set_crs("EPSG:4326", inplace=True)

population_centroids_gdf.to_file(data_temp + "population_centroids.gpkg", driver="GPKG")

In [ ]:
from pathlib import Path
from shapely.geometry import Polygon

In [ ]:
def read_tif(file: Path):
    if not file.exists():
        raise FileNotFoundError(f'File {file} not found')

    with rasterio.open(file) as dataset:
        arr = dataset.read()  # (bands X height X width)
        nodata = dataset.nodata
        transform = dataset.transform
        crs = dataset.crs

    # Replace NoData value with NaN
    if nodata is not None:
        arr[arr == nodata] = np.nan

    return arr.transpose((1, 2, 0)), transform, crs

def raster2vector(arr, transform, crs) -> gpd.GeoDataFrame:
    height, width, bands = arr.shape

    # Generate pixel coordinates
    geometries = []
    pixel_values = []

    for row in range(height):
        for col in range(width):
            x_min, y_max = transform * (col, row)  # Top-left corner
            x_max, y_min = transform * (col + 1, row + 1)  # Bottom-right corner

            pixel_value = arr[row, col].tolist()[0]  # Convert numpy array to list
            polygon = Polygon([(x_min, y_max), (x_max, y_max), (x_max, y_min), (x_min, y_min)])

            geometries.append(polygon)
            pixel_values.append(pixel_value)

    # Convert to DataFrame
    gdf = gpd.GeoDataFrame({'pop_grid_pop': pixel_values, 'geometry': geometries}, crs=crs)

    return gdf

epsg = 'EPSG:32632'

In [ ]:
# Preparing grid
grid_file = data_inputs + '100mGrid.gpkg'
grid = gpd.read_file(grid_file)
grid = grid.to_crs(epsg)
grid['grid_id'] = range(len(grid))
grid = grid[['grid_id', 'geometry','rowid', 'latitude', 'lat_min', 'lat_max', 'longitude', 'lon_min','lon_max']].set_geometry('geometry')
grid

In [ ]:
# Count buildings per grid cell

# Loading Google building footprints
building_file = data_inputs + 'Nairobi_GOBv3.gpkg'
buildings = gpd.read_file(building_file)
buildings = buildings.to_crs(epsg)
buildings['centroid'] = buildings['geometry'].centroid

# Joining buildings to grid
grid_buildings = grid.sjoin(buildings.set_geometry('centroid').drop(columns='geometry'), how='inner', predicate='intersects')
grid_buildings = grid_buildings.groupby('grid_id')

# Counting buildings per grid
building_counts = grid_buildings.size().rename('bcount')

# Adding building count to grid cells
grid = grid.merge(building_counts, on='grid_id', how='left')

# Assign building count 0 to cells with no buildings (NaN)
grid['bcount'] = grid['bcount'].fillna(0)
grid

In [ ]:
# Adding population data at 1km grid to finer grid
from pathlib import Path

data_path = Path(data_inputs)

# Loading coarse pop data
pop_file = data_path / 'nairobi_nga_f_15_49_2015_1km.tif'
pop_raster, transform, crs = read_tif(pop_file)

# Converting the raster grid to vector data
pop_grid = raster2vector(pop_raster, transform, crs)
pop_grid = pop_grid.to_crs(epsg)
pop_grid['pop_grid_id'] = range(len(pop_grid))

# Assign coarse population data to finer grid based on the centroid locations of the finer grid cells
grid['centroid'] = grid['geometry'].centroid
grid = gpd.sjoin(grid.set_geometry('centroid'), pop_grid, how='left', predicate='within')
print(grid.columns)
grid = grid[['grid_id', 'bcount', 'pop_grid_id', 'geometry','rowid', 'latitude', 'lat_min', 'lat_max',
       'longitude', 'lon_min', 'lon_max']]
grid.head()

In [ ]:
# Calculate population weight (fraction of total population count that should be assigned to cell based on its building count)
grid_grouped_pop = grid.groupby('pop_grid_id')
building_count_pop = grid_grouped_pop['bcount'].sum().rename('pop_grid_bcount')
grid = grid.merge(building_count_pop, on='pop_grid_id', how='left')
grid['pop_weight'] = grid['bcount'] / grid['pop_grid_bcount']

# Compute disaggregated population count based on weight and building count at coarser cell level
grid = grid.merge(pop_grid, on='pop_grid_id', how='left')
grid['pop'] = grid['pop_grid_pop'] * grid['pop_weight']
grid.head()

In [ ]:
# Saving to file
grid = grid.drop(columns=["geometry_y"])
grid.head()

In [ ]:
grid = grid.set_geometry("geometry_x")
grid = grid.to_crs(4326)
grid.to_file(data_temp + 'pop-grid-nairobi-centroids.gpkg', driver='GPKG')

In [ ]:
origin_gdf = population_centroids_gdf
origin_name_column = 'grid_code'
destination_gdf = healthcare_facilities_validated.dropna(subset=['geometry'])
destination_name_column = 'facility_name'

In [ ]:
origins = list(zip(origin_gdf.geometry.x, origin_gdf.geometry.y))

In [ ]:
destinations = list(zip(destination_gdf.geometry.x, destination_gdf.geometry.y))

In [ ]:
locations = origins + destinations

In [ ]:
origins_index = list(range(0, len(origins)))
destinations_index = list(range(len(origins), len(locations)))

In [ ]:
import requests

In [ ]:
body = {'locations': locations,
       'destinations': destinations_index,
       'sources': origins_index,
       'metrics': ['distance', 'duration']}

headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Authorization': api_key,
    'Content-Type': 'application/json; charset=utf-8'
}

response = requests.post('https://api.openrouteservice.org/v2/matrix/driving-car', json=body, headers=headers)

In [ ]:
distances = response.json().get('distances', [])
durations = response.json().get('durations', [])

In [ ]:
distances_duration_matrix = []

# Iterate over each origin (grid)
for origin_index, origin in origin_gdf.iterrows():
    origin_name = origin[origin_name_column]
    origin_x = origin.geometry.x
    origin_y = origin.geometry.y
    origin_distances = distances[origin_index]
    origin_durations = durations[origin_index]

    # find the minimum duration and the index of the minimum duration
    min_duration = min(origin_durations)
    min_index = origin_durations.index(min_duration)
    destination_index = destinations_index[min_index]
    dest_x, dest_y = locations[destination_index]
    filtered = healthcare_facilities_validated[(destination_gdf.geometry.x == dest_x) & (destination_gdf.geometry.y == dest_y) ]
    destination_row = filtered.iloc[0]
    dest_name = destination_row[destination_name_column]

        # Append both the distance and duration for this origin-destination pair
    distances_duration_matrix.append([
            origin_name, origin_y, origin_x,
            dest_name, dest_y, dest_x,
            min_duration
        ])

# Convert the results into a DataFrame
matrix_df = pd.DataFrame(distances_duration_matrix, columns=[
    'grid_code','origin_lat', 'origin_lon',
    'destination_name', 'dest_lat', 'dest_lon','min_duration'
])

In [ ]:
# Save to CSV
merged_df = pd.merge(matrix_df, grid_df[['grid_code', 'population']], on='grid_code', how='left')
merged_df.to_csv(data_temp + 'distance_duration_matrix_temp.csv', index=False)

In [ ]:
geometry = [Point(xy) for xy in zip(merged_df['dest_lon'], merged_df['dest_lat'])]
gdf = gpd.GeoDataFrame(merged_df, geometry=geometry, crs="EPSG:4326")

gpkg_path = data_temp + 'distance_duration_matrix_temp.gpkg'
gdf.to_file(gpkg_path, layer="duration_matrix", driver="GPKG")

In [ ]:
# If not loaded yet, read from the temporary folder
centroids_df = gpd.read_file(data_temp +'pop-grid-nairobi-centroids.gpkg')
centroids_df

In [ ]:
# If not loaded yet, read from the temporary folder
matrix_df = pd.read_csv(data_temp + 'OD-matrix-nairobi-access-emoc.csv')
matrix_df

In [ ]:
pop_centroids_hcf = pd.merge(matrix_df, centroids_df[['grid_id', 'bcount', 'pop_grid_bcount', 'pop_grid_pop', 'pop', 'geometry']], 
                     left_on='origin_id', right_on='grid_id', how='left')
pop_centroids_hcf

In [ ]:
distances_duration_matrix = pd.merge(pop_centroids_hcf, 
                                     healthcare_facilities_validated[['fid', 'Facility_n', 'Facility_t', 'Ownership', 
                                                                      'Lat', 'Long', 'Level', 'Offer EmOC (Y/N)',
                                                                      'Total In-patient beds', 'Maternity beds', 
                                                                      'Maternity theatres', 'Maternity beds', 
                                                                      'Maternity theatres', 'Operating time', 
                                                                      'Type of EmOC (basic/comprehensive)', 'geometry']], 
                     left_on='destination_id', right_on='fid', how='left')
distances_duration_matrix

In [ ]:
def local_validation(row):
    if row["Ownership"] in ["Local authority", "MoH"] and row["Type of EmOC (basic/comprehensive)"] == "BMOC":
        return "Public Basic EmOC"
    elif row["Ownership"] in ["Local authority", "MoH"] and row["Type of EmOC (basic/comprehensive)"] != "BMOC":
        return "Public Comprehensive EmOC"
    elif row["Ownership"] not in ["Local authority", "MoH"] and row["Type of EmOC (basic/comprehensive)"] == "BMOC":
        return "Private Basic EmOC"
    elif row["Ownership"] not in ["Local authority", "MoH"] and row["Type of EmOC (basic/comprehensive)"] != "BMOC":
        return "Private Comprehensive EmOC"
    else:
        return "Other"

distances_duration_matrix["Local_Validation"] = distances_duration_matrix.apply(local_validation, axis=1)

In [ ]:
# Step 1: creat subsets based on categories of 'Validation of HCFs Categorization'
categories = {
    "public_comprehensive_EmOC": ["Public Comprehensive EmOC"],
    "private_comprehensive_EmOC": ["Private Comprehensive EmOC"],
    "private_basic_EmOC": ["Private Basic EmOC"],
    "public_basic_EmOC": ["Public Basic EmOC"]
}

subsets = {
    key: distances_duration_matrix[
        distances_duration_matrix['Local_Validation'].str.contains('|'.join(values), na=False)
    ]
    for key, values in categories.items()
}

public_CEmOC = subsets["public_comprehensive_EmOC"]
private_CEmOC = subsets["private_comprehensive_EmOC"]
public_BEmOC = subsets["public_basic_EmOC"]
private_BEmOC = subsets["private_basic_EmOC"]

In [ ]:
# Step 2: Define a function to get 3 smallest duration_seconds per grid_id for each category
def get_closest_3(df, n=3):
    return df.groupby('grid_id').apply(lambda x: x.nsmallest(n, 'duration_seconds')).reset_index(drop=True)
                      
# If the subsets are already created for each category, we apply the function to each subset:
public_CEmOC_closest_3 = get_closest_3(public_CEmOC)
private_CEmOC_closest_3 = get_closest_3(private_CEmOC)
public_BEmOC_closest_3 = get_closest_3(public_BEmOC)
private_BEmOC_closest_3 = get_closest_3(private_BEmOC)

# Step 4: Concatenate the filtered results into a single DataFrame
distances_duration_matrix = pd.concat([
    public_CEmOC_closest_3, private_CEmOC_closest_3,
    public_BEmOC_closest_3, private_BEmOC_closest_3
])

In [ ]:
distances_duration_matrix = distances_duration_matrix.rename(columns={
    "geometry_x": "geometry",
    "Long": "longitude_hcf",
    "Lat": "latitude_hcf",
    "Facility_n": "facility_name",
    "pop": "population",
    "Local_Validation": "local_validation",
    "geometry_y": "geometry_hcf",
    "fid": "hcf_id"
})
columns_to_keep = ["grid_id", "hcf_id",  "longitude_hcf", "latitude_hcf", "facility_name", "local_validation", 
                   "population", "duration_seconds", "distance_km", "geometry"]
pop_centroids_closest_hcf = distances_duration_matrix[columns_to_keep]

In [ ]:
origin_dest = pop_centroids_closest_hcf

In [ ]:
# Function
d = 10 * 60 # try max duration 10mins/30mins car, under estimation of travel time and traffic condition realted to the selected data sourse 
W = 0.5
beta = - d ** 2 / log(W)
print(beta)

In [ ]:
# Convert 'duration' to numeric, coercing errors to NaN
origin_dest['duration_seconds'] = pd.to_numeric(origin_dest['duration_seconds'], errors='coerce')

# Drop rows with NaN values in 'duration' column
origin_dest = origin_dest.dropna(subset=['duration_seconds'])
origin_dest['grid_id'] = pd.to_numeric(origin_dest['grid_id'], errors='coerce')

origin_dest_acc = origin_dest  # Backup

In [ ]:
import math

In [ ]:
# Apply Gaussian decay function to calculate the weight of each grid to healthcare 
# facilities based on the travel duration. d is the travel time and beta is the decay 
# parameter previously calculated.
# The weight decreases as the duration increases, meaning facilities that are further away have less impact.
origin_dest_acc['Weight'] = origin_dest_acc['duration_seconds'].apply(lambda d: round(math.exp(-d**2/beta), 8))

In [ ]:
# Compute the Weighted Population (Pop_W), the population of each grid cell is multiplied 
# by the corresponding weight to calculate the weighted population.
origin_dest_acc['Pop_W'] = origin_dest_acc['population'] * origin_dest_acc['Weight']

In [ ]:
# Sum the Weighted Population
origin_dest_sum = origin_dest_acc.groupby(by='hcf_id')['Pop_W'].sum().reset_index()

In [ ]:
# Merge the Sum of Weighted Population Back into the Original Data
origin_dest_acc = origin_dest_acc.merge(origin_dest_sum, on='hcf_id')

In [ ]:
# Suppy is based on the categories of ownership and service level of the healthcare facilities
supply_map = {
    'Public Comprehensive EmOC': 1,
    'Private Comprehensive EmOC': 0.7,
    'Public Basic EmOC': 0.5,
    'Private Basic EmOC': 0.35
}

In [ ]:
# Compute the Supply-Demand Ratio (Rj)
origin_dest_acc['supply'] = origin_dest_acc['local_validation'].map(supply_map)
origin_dest_acc['supply_demand_ratio'] = 1 / origin_dest_acc.Pop_W_S
origin_dest_acc['supply_demand_ratio'].replace([np.inf, np.nan], 0, inplace=True)

In [ ]:
# Calculate Rj * Weight for Each Grid Cell
origin_dest_acc['supply_W'] = origin_dest_acc['supply_demand_ratio'] * origin_dest_acc.Weight

In [ ]:
# Compute Accessibility Index (Ai) for Each Grid Cell
origin_dest_acc['Accessibility'] = origin_dest_acc.groupby('grid_id')['supply_W'].transform('sum')

In [ ]:
! pip install scikit-learn
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Normalize

scaler = MinMaxScaler()
origin_dest_acc['Accessibility_standard'] = scaler.fit_transform(origin_dest_acc[['Accessibility']])

In [ ]:
max(origin_dest_acc.Accessibility_standard)

In [ ]:
origin_dest_acc_gdf = gpd.GeoDataFrame(origin_dest_acc, geometry='geometry', crs="EPSG:4326")
gpkg_path = data_temp + 'acc_score_3closest.gpkg'
origin_dest_acc_gdf.to_file(gpkg_path, layer="acc_score_3closest", driver="GPKG")

In [ ]:
# Read the GeoPackage file (if starting from this section)
results_grid = gpd.read_file(data_temp + 'acc_score_3closest.gpkg')

In [ ]:
# Group by multiple columns and calculate the mean for numeric columns
# results_grid = results_grid.groupby(['grid_id', 'origin_lon', 'origin_lat', 'origin_lon_min', 'origin_lat_min', 'origin_lon_max', 'origin_lat_max', 'Accessibility_standard']).count().reset_index()

results_grid = results_grid.drop_duplicates(['grid_id', 'Accessibility_standard', 'geometry'])

type(results_grid)

In [ ]:
results_grid['result'] = -1
results_grid.loc[results_grid['Accessibility_standard'] > 0, 'result'] = 2
results_grid.loc[results_grid['Accessibility_standard'] > pow(10, -1.3605), 'result'] = 1
results_grid.loc[results_grid['Accessibility_standard'] > pow(10, -1.1293), 'result'] = 0


In [ ]:
category_counts = results_grid['result'].value_counts()
print(category_counts)

In [ ]:
category_counts = results_grid['focused'].value_counts()
print(category_counts)

In [ ]:
results_grid['focused'] = 0
# Focus areas between the High and medium categories
results_grid.loc[(results_grid['Accessibility_standard'] < pow(10, -1.26)) & (results_grid['Accessibility_standard'] > pow(10, -1.46)), 'focused'] = 1

results_grid

In [ ]:
results_grid = results_grid.loc[results_grid['result'] != -1]

In [ ]:
# Extract bounding box coordinates and create columns for min and max coordinates
results_grid['origin_lon_min'] = results_grid.geometry.apply(lambda geom: geom.bounds[0])
results_grid['origin_lat_min'] = results_grid.geometry.apply(lambda geom: geom.bounds[1])
results_grid['origin_lon_max'] = results_grid.geometry.apply(lambda geom: geom.bounds[2])
results_grid['origin_lat_max'] = results_grid.geometry.apply(lambda geom: geom.bounds[3])

In [ ]:
# Calculate centroid coordinates for each geometry and add as columns
results_grid['longitude'] = results_grid.geometry.centroid.x
results_grid['latitude'] = results_grid.geometry.centroid.y

In [ ]:
results_grid = results_grid.rename(columns={
    'origin_lon': 'longitude',
    'origin_lat': 'latitude',
    'origin_lon_min': 'lon_min',
    'origin_lat_min': 'lat_min',
    'origin_lon_max': 'lon_max',
    'origin_lat_max': 'lat_max'
})

results_grid

In [ ]:
# Save the results to a new GeoPackage file
output_gpkg_path = data_temp + 'emergency-maternal-care-deprivation-access.gpkg'
results_grid.to_file(output_gpkg_path, layer='emergency-maternal-care-deprivation-access', driver='GPKG')


In [ ]:
 #Save the results to a CSV file in the format required by the IDEAMAPS data ecosystem
columns_to_keep = ["longitude", "latitude",  "lon_min", "lat_min", "lon_max", "lat_max", 
                   "focused", "result",]
results_table = results_grid[columns_to_keep]

results_table.to_csv(data_outputs + 'model-output.csv', index=False)